In [1]:
%matplotlib inline
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import scipy.stats as ss
import seaborn as sns
import numpy as np
import warnings

import scipy.stats as ss
warnings.filterwarnings('ignore')
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

In [2]:
def loadDF(filename, sep = None):
    with open(filename, 'rb') as inp:
        out = inp.read().decode()
    out = out.split()
    out = pd.DataFrame([i.strip(sep).split(sep) for i in out[1:]], columns = [i for i in out[0].split(sep)])
    return out

#### Cargo el DataFrame

In [4]:
df2011 = loadDF('./data/Base Usuario ENPreCoSP-2011.txt', '|')
df2008 = loadDF('./data/ENPreCoSP-2008 Base Usuario.txt', ';')

In [5]:
df2011.info()
df2011.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34343 entries, 0 to 34342
Columns: 291 entries, IDENTIFI to BIDT03
dtypes: object(291)
memory usage: 76.2+ MB


,IDENTIFI,W_PERS,REGION,POB_URB,PRVNC,AGL_URB,BHCV01,BHCV02,BHCV03,BHCV04,...,BIDT02_05,BIDT02_06,BIDT02_07,BIDT02_08,BIDT02_09,BIDT02_10,BIDT02_11,BIDT02_12,BIDT02_13,BIDT03
0,61301445099,3787,1,4,6,1,3,4,1,1,...,None,None,None,None,None,None,None,None,None,None
1,94924403348,5301,1,4,6,1,3,4,1,1,...,None,None,None,None,None,None,None,None,None,None
2,63798413037,2248,1,4,6,1,3,4,1,1,...,None,None,None,None,None,None,None,None,None,None
3,75477993771,6157,1,4,6,1,3,3,1,1,...,None,None,None,None,None,None,None,None,None,None
4,34764150916,12528,1,4,6,1,3,4,1,1,...,None,None,None,None,None,None,None,None,None,None


In [23]:
df2008.info()
df2008.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34203 entries, 0 to 34202
Columns: 296 entries, IDENTIFI to Año
dtypes: object(296)
memory usage: 77.2+ MB


,IDENTIFI,W_PERS,REGION,POB_URB,PROV,AGL_URB,BHCV01,BHCV02,BHCV03,BHCV04,...,BIDT02_06,BIDT02_07,BIDT02_08,BIDT02_09,BIDT02_10,BIDT02_11,BIDT02_12,BIDT02_13,BIDT03,Año
0,25662899,3108,2,3,6,8,1,1,1,1,...,None,None,None,None,None,None,None,None,None,2008
1,77993054,3238,2,3,6,8,1,2,2,2,...,None,None,None,None,None,None,None,None,None,2008
2,98825711,1415,2,3,6,8,1,2,2,1,...,None,None,None,None,None,None,None,None,None,2008
3,58165012,1114,2,3,6,8,2,2,2,1,...,None,None,None,None,None,None,None,None,None,2008
4,80083154,1573,2,3,6,8,1,3,3,2,...,None,None,None,None,None,None,None,None,None,2008


In [24]:
#No hay sujetos repetidos
set(df2008['IDENTIFI'].unique()) & set(df2011['IDENTIFI'].unique())

set()

In [25]:
df2008['Año'] = '2008'
df2011['Año'] = '2011'

In [26]:
#Columnas de diferencia
#Por ahora saco todas, hay que revisar

print(df2011.columns.difference(df2008.columns),
      df2008.columns.difference(df2011.columns))
lio = df2011.columns.difference(df2008.columns).union(
      df2008.columns.difference(df2011.columns))
df2008_drop = df2008.drop(df2008.columns.intersection(lio), axis = 1)
df2011_drop = df2011.drop(df2011.columns.intersection(lio), axis = 1)

Index(['BHHO01', 'BHHO02', 'BHHO03', 'BIOD01_01', 'BIOD01_02', 'BIOD01_03',
       'BIOD02_01', 'BIOD02_02', 'BIOD02_03', 'BIOD03_01', 'BIOD03_02',
       'BIOD03_03', 'BIOD04_01', 'BIOD04_02', 'BIOD04_03', 'BIOD05_01',
       'BIOD05_02', 'BIOD05_03', 'PRVNC'],
      dtype='object') Index(['BHCV12', 'BHCV13', 'BHCV14', 'BIODA_01', 'BIODA_02', 'BIODA_03',
       'BIODB_01', 'BIODB_02', 'BIODB_03', 'BIODC_01', 'BIODC_02', 'BIODC_03',
       'BIODD_01', 'BIODD_02', 'BIODD_03', 'BIODE_01', 'BIODE_02', 'BIODE_03',
       'DEP_CO', 'DEP_EX', 'DEP_MA', 'DEP_PB', 'PROV'],
      dtype='object')


In [27]:
#Uno en un DataFrame

dftotal = pd.concat([df2008_drop,df2011_drop], ignore_index=True)
dftotal.shape

(68546, 273)

In [28]:
#Sampleo para facilitar

#dftotal = dftotal.sample(dftotal.shape[0]//2)

### Limpieza

In [29]:
%%time

dftotal = dftotal.replace('^\s*$', np.nan, regex=True)
dftotal = dftotal.apply(lambda x: pd.to_numeric(x, errors = 'ignore'))

CPU times: user 10.9 s, sys: 268 ms, total: 11.2 s
Wall time: 11.2 s


In [30]:
def limpiar_col(c):
    c[(c <= c.quantile(0.0001))] = c.quantile(0.0001)
    c[(c >= c.quantile(0.9999))] = c.quantile(0.9999)
    return c

In [31]:
%%time

dftotal = dftotal.apply(limpiar_col, axis = 0)

CPU times: user 3.42 s, sys: 172 ms, total: 3.59 s
Wall time: 2.16 s


In [32]:
dftotal.head()

,IDENTIFI,W_PERS,REGION,POB_URB,AGL_URB,BHCV01,BHCV02,BHCV03,BHCV04,BHCV05,...,BIDT02_06,BIDT02_07,BIDT02_08,BIDT02_09,BIDT02_10,BIDT02_11,BIDT02_12,BIDT02_13,BIDT03,Año
0,25662899.0,3108.0,2,3,8,1,1.0,1,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008
1,77993054.0,3238.0,2,3,8,1,2.0,2,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008
2,98825711.0,1415.0,2,3,8,1,2.0,2,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008
3,58165012.0,1114.0,2,3,8,2,2.0,2,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008
4,80083154.0,1573.0,2,3,8,1,3.0,3,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008


In [33]:
def tabaco(row):
    if (row['BITA05'] > 10 ) | (row['BITA06'] > 0 ):
        return 1
    else:
        return 0
def marihuana(row):
    if sum([row['BIMA04'] == 1, row['BIMA06'] == 1, row['BIMA11'] == 1,row['BIMA12'] == 1,
        row['BIMA09'] == 1, row['BIMA13'] == 1, row['BIMA16'] == 1]):
        return 1
    else:
        return 0
def alcohol(row):
    if ((row['AB_TC'] == 1) | (row[['BIBA07_01', 'BIBA07_02', 'BIBA07_03']].sum() > 20) |
    (row[['BIBA08_01', 'BIBA08_02', 'BIBA08_03', 'BIBA09_01', 'BIBA09_02', 'BIBA09_03']].sum() > 20) |
    (row['BIBA12'] == 1) | (row['BIBA06'] >= 3) | (row['BIBA11'] > 9)): 
        return 1
    else:
        return 0
def cocaina(row):
    if ((row['BICO04'] == 1) | (row['BICO06'] == 1) | (row['BICO11'] == 1) | 
        (row['BICO12'] == 1) | (row['BICO13'] == 1) | (row['BICO14'] == 1) | (row['BICO16'] == 1)):
        return 1
    else:
        return 0
def pastabase(row):
    if ((row['BIPB04'] == 1) | (row['BIPB06'] == 1) | (row['BIPB11'] == 1) | 
        (row['BIPB12'] == 1) | (row['BIPB13'] == 1) | (row['BIPB14'] == 1) | (row['BIPB16'] == 1)):
        return 1
    else:
        return 0
def otras(row):
    if ((row['BIEX04'] == 1) | (row['BIEX06'] == 1) | (row['BIEX11'] == 1) | 
        (row['BIEX12'] == 1) | (row['BIEX13'] == 1) | (row['BIEX15'] == 1) | 
        (row['BIEX14'] == 1) | (row['BIEX16'] == 1) | (row['BIES08'] >= 3) | 
        (row['BITR08'] >= 10) | (row['BIIN01'] == 1)):
        return 1
    else:
        return 0
def tratamientos(row):
    if ((row['BIDT01'] == 1) | (row['BIDT03'] in {1, 2})):
        return 1
    else:
        return 0

In [34]:
tar = pd.DataFrame()
tar['Tabaco'] = dftotal.apply(tabaco, axis = 1)
tar['Marihuana'] = dftotal.apply(marihuana, axis = 1)
tar['Cocaina'] = dftotal.apply(cocaina, axis = 1)
tar['Alcohol'] = dftotal.apply(alcohol, axis = 1)
tar['Otras'] = dftotal.apply(otras, axis = 1)
tar['Tratamiento'] = dftotal.apply(tratamientos, axis = 1)

In [35]:
tar.sample(5)

,Tabaco,Marihuana,Cocaina,Alcohol,Otras,Tratamiento
63995,0,0,0,0,0,0
30786,1,0,0,0,0,0
16261,0,0,0,0,0,0
26736,0,0,0,0,0,0
25817,1,0,0,0,0,0


In [36]:
dftotal.head()

,IDENTIFI,W_PERS,REGION,POB_URB,AGL_URB,BHCV01,BHCV02,BHCV03,BHCV04,BHCV05,...,BIDT02_06,BIDT02_07,BIDT02_08,BIDT02_09,BIDT02_10,BIDT02_11,BIDT02_12,BIDT02_13,BIDT03,Año
0,25662899.0,3108.0,2,3,8,1,1.0,1,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008
1,77993054.0,3238.0,2,3,8,1,2.0,2,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008
2,98825711.0,1415.0,2,3,8,1,2.0,2,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008
3,58165012.0,1114.0,2,3,8,2,2.0,2,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008
4,80083154.0,1573.0,2,3,8,1,3.0,3,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008


In [37]:
#Desde el 62 son cuestionarios
dftotal.iloc[:,:62][dftotal['BITR01'] == 2].count()[0]

53574

In [38]:
bitr = [c for c in dftotal.columns if c[:4] == 'BITR']
dftotal[dftotal['BITR01'] == 2].loc[:,bitr].head()

,BITR01,BITR02,BITR03,BITR04,BITR05,BITR06,BITR07_01,BITR07_02,BITR07_03,BITR07_04,BITR07_05,BITR07_09,BITR08,BITR09_01,BITR09_02
0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,3
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3
3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,4
4,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,4


### Modelado

In [40]:
#Importamos las funciones que usaremos
!pip install xgboost
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV, train_test_split, StratifiedKFold
from sklearn.pipeline import make_pipeline
import xgboost

     |████████████████████████████████| 109.8MB 11.0MB/s eta 0:00:01


In [41]:
#Devuelve las n features más importantes, con nombre y ordenadas
def get_most_important(X, model, n):
    return sorted(zip(X.columns, 
                      [str(round(i, 4)) for i in model.feature_importances_]), key = lambda x: x[1], reverse = True)[:n]

In [42]:
#funcion para generar un modelo para cada droga
def make_model(drug, params, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify = df_target)
    model = RandomForestClassifier(**params)
    model.fit(X_train, y_train[drug])
    print('Cross val score: ', cross_val_score(model, X_test, y_test[drug]).mean())
    print('Roc score:', roc_auc_score(y_test[drug], model.predict_proba(X_test)[:,1]))
    print(classification_report(y_test[drug], model.predict(X_test)))
    return model

In [43]:
#Un wrapper sobre gridsearchCV. Uso recall de scoring para aumentar los positivos detectados
def get_best(model, params, drug, X, y, scoring = None):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify = y[drug])
    kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=11)
    param_search = RandomizedSearchCV(model,params, 
                              cv = kf, n_jobs = -1, tverbose = 1, scoring = 'recall')

    param_search.fit(X, y[drug])
    print('Best score: {}'.format(param_search.best_score_))
    print('Best parameters: {}'.format(param_search.best_params_))
    try:
        print('Roc: {}'.format(roc_auc_score(y_test[drug], 
                                             param_search.best_estimator_.predict_proba(X_test)[:,1])))
    except:
        pass
    print(classification_report(y_test[drug], param_search.predict(X_test)))
    return param_search.best_estimator_

In [44]:
nsnc = ['9', '99', '999', '888']
features = [c for c in dftotal.columns][2:62] + ['BITA03', 'BITA09_01', 'BITA09_02', 'BIBA03', 'BIBA14_01', 'BIBA14_02',
            'BITR01','BITR05', 'BITR06', 'BITR09_01']
X = pd.get_dummies(dftotal.loc[:, features])

In [45]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, tar, test_size = 0.3)

In [46]:
xgb = xgboost.XGBClassifier()

In [47]:
params_xgb = {'n_estimators': [25, 50, 75, 100, 125, 150, 160, 170, 180, 190]}
#300

In [48]:
params_xgb = {'n_estimators': [5, 20, 30,  50, 100, 125, 150, 175, 190, 200],
              'learning_rate': [1/i for i in range(1, 100, 3)],
              'reg_alpha':[1e-10, 1e-5, 1e-2, 0.1, 1, 10, 20, 50, 100]}
 
'''
'max_depth': range(3,10,1)
'min_child_weight':[1, 2, 4, 6, 8, 10, 12],
 'gamma':[1/i for i in range(1,100)],
 'subsample':[1/i for i in range(1,100)],
 'colsample_bytree':[1/i for i in range(1,50)], 
'''

"\n'max_depth': range(3,10,1)\n'min_child_weight':[1, 2, 4, 6, 8, 10, 12],\n 'gamma':[1/i for i in range(1,100)],\n 'subsample':[1/i for i in range(1,100)],\n 'colsample_bytree':[1/i for i in range(1,50)], \n"

In [49]:
%%time
marihuana = get_best(xgb, params_xgb, 'Marihuana', Xtrain, ytrain)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   43.8s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.4min finished


Best score: 0.19184134631143884
Best parameters: {'reg_alpha': 10, 'n_estimators': 200, 'learning_rate': 0.03571428571428571}
Roc: 0.9906080101486047
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     14240
           1       0.80      0.26      0.39       155

    accuracy                           0.99     14395
   macro avg       0.90      0.63      0.69     14395
weighted avg       0.99      0.99      0.99     14395

CPU times: user 55.8 s, sys: 731 ms, total: 56.6 s
Wall time: 1min 33s


In [50]:
marihuana

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.03571428571428571, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=200, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=10,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

In [51]:
%%time
alcohol = get_best(xgb, params_xgb, 'Alcohol', Xtrain, ytrain)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  2.3min finished


Best score: 0.8805327726521394
Best parameters: {'reg_alpha': 1, 'n_estimators': 190, 'learning_rate': 0.011764705882352941}
Roc: 0.8544503522176115
              precision    recall  f1-score   support

           0       0.87      0.66      0.75      7617
           1       0.70      0.89      0.79      6778

    accuracy                           0.77     14395
   macro avg       0.79      0.78      0.77     14395
weighted avg       0.79      0.77      0.77     14395

CPU times: user 52.9 s, sys: 614 ms, total: 53.5 s
Wall time: 2min 22s


In [52]:
%%time
tabaco = get_best(xgb, params_xgb, 'Tabaco', Xtrain, ytrain)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  2.3min finished


Best score: 0.8556312635357513
Best parameters: {'reg_alpha': 50, 'n_estimators': 125, 'learning_rate': 0.010638297872340425}
Roc: 0.8888705547231083
              precision    recall  f1-score   support

           0       0.92      0.78      0.85     10114
           1       0.63      0.85      0.72      4281

    accuracy                           0.80     14395
   macro avg       0.78      0.82      0.78     14395
weighted avg       0.84      0.80      0.81     14395

CPU times: user 40 s, sys: 410 ms, total: 40.5 s
Wall time: 2min 24s


In [53]:
%%time
cocaina = get_best(xgb, params_xgb, 'Cocaina', Xtrain, ytrain)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.4min finished


Best score: 0.04280440392419885
Best parameters: {'reg_alpha': 1e-10, 'n_estimators': 50, 'learning_rate': 0.0625}
Roc: 0.9608523459572036
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     14346
           1       0.92      0.24      0.39        49

    accuracy                           1.00     14395
   macro avg       0.96      0.62      0.69     14395
weighted avg       1.00      1.00      1.00     14395

CPU times: user 12.1 s, sys: 191 ms, total: 12.3 s
Wall time: 1min 25s


In [54]:
tratamiento = get_best(xgb, params_xgb, 'Tratamiento', Xtrain, ytrain)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.5min finished


Best score: 0.026125996000728437
Best parameters: {'reg_alpha': 1e-10, 'n_estimators': 5, 'learning_rate': 1.0}
Roc: 0.9174698813671174
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     14165
           1       0.84      0.09      0.16       230

    accuracy                           0.99     14395
   macro avg       0.91      0.55      0.58     14395
weighted avg       0.98      0.99      0.98     14395



In [ ]:
otras = get_best(xgb, params_xgb, 'Otras', Xtrain, ytrain)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
